In [1]:
import { requireCytoscape, requireCarbon } from "./lib/draw";

requireCarbon();
requireCytoscape();

# Interpreters

## Where Were We?

1. Language primitives (i.e., building blocks of languages)
2. Language paradigms (i.e., combinations of language primitives)
3. **Building a language** (i.e., designing your own language)
    * Last time: lexing and parsing.
    * Today: **interpreters** and **evaluation**.

## Goal

1. Last time we saw how to convert a string into an expression/AST.
2. Today we want to write a *program*, an **interpreter**, that converts an expression/AST into a **value**, i.e., an expression/AST that has no computation left to be run.

## LambdaTS Expression/AST

```ts
type Expr = NumericConstant | BinaryExpr | ConditionalExpr | FunctionExpr | Identifier | CallExpr;
type BinaryExpr = { tag: "BINARY";
    operator: BinaryOperator;
    left: Expr;
    right: Expr;
};
// Note (e) is not part of the AST
type ConditionalExpr = { tag: "CONDITIONAL";
    condExpr: Expr;
    thenExpr: Expr;
    elseExpr: Expr;
};
type FunctionExpr = { tag: "FUNCTION";
    parameter: string;
    body: Expr;
};
type CallExpr = { tag: "CALL";
    func: Expr;
    argument: Expr;
};
```

## Interpreter for LambdaTS by Example

- We'll introduce an interpreter by example.
- First we need our `drawProg` function.

In [2]:
import { draw, treeLayout } from "./lib/draw";
import * as T from "./lib/lambdats/token";
import * as E from "./lib/lambdats/expr";
import * as Parser from "./lib/lambdats/parser";

function drawProg(prog: string|E.Expr): void {
    if (typeof prog === 'string') {
        draw(E.cytoscapify(Parser.parse(prog)), 800, 350, treeLayout);
    } else {
        draw(E.cytoscapify(prog), 800, 350, treeLayout);
    }
}

### Example 1

In [3]:
// The input / output relation should match the TypeScript interpreter
const input = 1 + 2 ? ((x) => 1) : ((x) => 2)
const output = input.toString()
output

(x) => 1


#### Desired LambdaTS intepreter Output

Input:
```ts
1 + 2 ? λx => 1 : λx => 2
```

Output:
```ts
λx => 1
```

In [4]:
// Input AST
drawProg("1 + 2 ? λx => 1 : λx => 2")

In [5]:
// Output AST
drawProg("λx => 1")

### Example 2

In [6]:
// The input / output relation should match the TypeScript interpreter
const input = ((x) => 1 + 1 + 1)((x) => 1 + 1 + 1)
const output = input.toString()
output

3


#### Desired LambdaTS intepreter Output

Input:
```ts
(λx => 1 + 1 + 1)(λx => 1 + 1 + 1)
```

Output:
```ts
3
```

In [7]:
// Input AST
drawProg("(λx => 1 + 1 + 1)(λx => 1 + 1 + 1)")

In [8]:
// Output AST
drawProg("3")

### Example 3

In [9]:
const input = 1 + ((x) => x + 1 + 1 + 1)

1:15 - Operator '+' cannot be applied to types 'number' and '(x: any) => any'.


#### Desired LambdaTS Interpreter Output

Input:
```ts
1 + (λx => 1 + 1 + 1)
```

Output: error

In [10]:
// Input AST
drawProg("1 + (λx => 1 + 1 + 1)")

In [11]:
// Output AST

## Implementing LambdaTS Interpreter

1. Define **values** formally.
2. Show interpreter on arithmetic subset of LambdaTS.
3. Show interpreter on all of LambdaTS where we need to interpret first-class functions. This section should give you a mental model for understanding variable binding.

### LambdaTS Values

1. Informally, a *value* is an expression/AST that has no more computation left to be run.
2. Before we can write an interpreter, we'll need to formally define what **values** are in a way that a computer can understand.
3. We'll see that values are a subset of expressions. Recall expressions below.

```ts
type Expr = NumericConstant | BinaryExpr | ConditionalExpr | FunctionExpr | Identifier | CallExpr;
```

In [12]:
// We pick the subset of expressions that correspond to numbers and functions
type Value = T.NumericConstant | E.FunctionExpr

#### That's It!

- We've identified a simple subset of expressions that are values.
- Let's see some examples now.

#### Example 1

In [13]:
drawProg("1"); // value and expression

#### Example 2

In [14]:
drawProg("λx => x"); // value and expression

#### Example 3

In [15]:
drawProg("λx => 1 + x"); // value and expression

#### Example 4

In [16]:
drawProg("λx => 1 + 1 + 1"); // value and expression

#### Non-Example 1

In [17]:
1 + 1

2


In [18]:
drawProg("1 + 1");

#### Non-Example 2

In [19]:
1 + ((x) => x)

1:1 - Operator '+' cannot be applied to types 'number' and '(x: any) => any'.


In [ ]:
drawProg("1 + (λx => x)");

#### Non-Example 3

In [20]:
((x) => 1 + x)(2)

3


In [21]:
drawProg("(λx => 1 + x)(2)");

#### Non-Example 4

In [22]:
(λx => 1 + 1 + 1)(λx => 1 + 1 + 1)

3


In [23]:
drawProg("(λx => 1 + 1 + 1)(λx => 1 + 1 + 1)");

#### Observations

1. Values refer to expressions (see `FunctionValue`).
2. Values form a *subset* of expressions, i.e., every value is an expression but not vice versa. (You may be wondering about environments. Recall that you can encode dictionaries with first-class functions.)

#### Examples

### Now it's time to write an interpreter!

Goal: reproduce what TypeScript already gives us.

```
function interpret(e: E.Expr): Value {
   // TODO
}
```

### Interpret with Arithmetic

Let's begin by focusing on just part of the language, the part that deals with arithmetic.

```ts
export type Expr = NumericConstant | BinaryExpr | ConditionalExpr;

export type BinaryExpr = { tag: "BINARY";
    operator: BinaryOperator;
    left: Expr;
    right: Expr;
};

export type ConditionalExpr = { tag: "CONDITIONAL";
    condExpr: Expr;
    thenExpr: Expr;
    elseExpr: Expr;
};
```

#### Idea: Recursively Reduce Expression to Value


```ts
interpert(1 + (4 / 2)) = interpert(1) + interpret(4 / 2)
                       = 1 + (interpret(4) / interpret(2))
                       = 1 + (4 / 2)
                       = 3
```

In [24]:
const inputAST = Parser.parse("1 + 4 / 2");
drawProg(inputAST);

In [25]:
// Left
const intermediateAST0 = Parser.parse("1");
drawProg(intermediateAST0);

In [26]:
// Right
const intermediateAST1 = Parser.parse("4 / 2");
drawProg(intermediateAST1);
const intermediateAST2 = Parser.parse("4");
drawProg(intermediateAST2);
const intermediateAST3 = Parser.parse("2");
drawProg(intermediateAST3);

In [27]:
// Output
const valueAndOutputAST = Parser.parse("3");
drawProg(valueAndOutputAST);

In [28]:
function interpret(e: E.Expr): Value {
    switch (e.tag) {
        case "NUMBER": {
            return T.mkNumericConstant(e.value);
        }
        case "BINARY": { // 1 + 2
            const leftVal = interpret(e.left);
            const rightVal = interpret(e.right);
            return interpretBinop(leftVal, e.operator, rightVal);
        }
        case "CONDITIONAL": {
            const condVal = interpret(e.condExpr);
            if (condVal.tag === "NUMBER" && condVal.value !== 0) {
                return interpret(e.thenExpr);
            } else {
                return interpret(e.elseExpr);
            }
        }
        default: {
            throw Error(`Tag ${e.tag} not supported yet ...` );
        }
    }
}

function interpretBinop(leftVal: Value, op: T.BinaryOperator, rightVal: Value): Value {
    if (leftVal.tag === "NUMBER" && rightVal.tag === "NUMBER") {
        switch (op) {
            case "+": {
                return T.mkNumericConstant(leftVal.value + rightVal.value);
            }
            case "-": {
                return T.mkNumericConstant(leftVal.value - rightVal.value);
            }
            case "*": {
                return T.mkNumericConstant(leftVal.value * rightVal.value);
            }
            case "/": {
                return T.mkNumericConstant(leftVal.value / rightVal.value);
            }
        }
    } else {
        throw Error(`Attempting ${leftVal} ${op} ${rightVal}`);
    }
}

In [29]:
const inputAST = Parser.parse("1 + 1");
const outputAST = interpret(inputAST);
console.log("Input");
drawProg(inputAST);
console.log("Output");
drawProg(outputAST);
console.log("Expected output", 1 + 1);

Input


Output


Expected output 2


In [30]:
const inputAST = Parser.parse("1 + 4 / 2");
const outputAST = interpret(inputAST);
console.log("Input");
drawProg(inputAST);
console.log("Output");
drawProg(outputAST);
console.log("Expected output", 1 + 4 / 2);

Input


Output


Expected output 3


In [31]:
const inputAST = Parser.parse("1 ? 0 : (2 + 1)");
const outputAST = interpret(inputAST);
console.log("Input");
drawProg(E.exprToString(inputAST));
console.log("Output");
drawProg(E.exprToString(outputAST));
console.log("Expected output", 1 ? 0 : (2 + 1));

Input


Output


Expected output 0


### Interpret with First-Class Functions

Main question: How do we interpret variables?

In [32]:
1 + x;  // Should have compile error, what does x refer to?

1:5 - Cannot find name 'x'.


In [ ]:
const x = 1 + 2;
const y = 2 + 3;
1 + x + y;   // substitute 3 for x and substitute 5 for y

#### Semantics of variables is substitution

1. A variable only has meaning when it is in scope.
2. When a variable is in scope, it's meaning is "substitute me" for whatever I was defined for.

In [33]:
function substitute(orig: E.Expr, x: string, other: E.Expr): E.Expr {
    switch (orig.tag) {
        case "NUMBER": {
            return T.mkNumericConstant(orig.value);
        }
        case "BINARY": {                
            return E.mkBinaryExpr(substitute(orig.left, x, other),
                                  orig.operator,
                                  substitute(orig.right, x, other));
        }
        case "CONDITIONAL": {
            return E.mkConditionalExpr(substitute(orig.condExpr, x, other),
                                       substitute(orig.thenExpr, x, other),
                                       substitute(orig.elseExpr, x, other));
        }
        case "FUNCTION": { 
            return orig.parameter === x ? orig : E.mkFunctionExpr(orig.parameter, substitute(orig.body, x, other));
        }
        case "IDENTIFIER": {
            return orig.name === x ? other : orig;
        }
        case "CALL": {
            return E.mkCallExpr(substitute(orig.func, x, other),
                                substitute(orig.argument, x, other));
        }
        default: {
            throw Error("Shouldn't happen");
        }
    }
}

In [34]:
const origAST = Parser.parse("x");
const otherAST = Parser.parse("1");
console.log("Original");
drawProg(origAST);
console.log("Other AST");
drawProg(otherAST);
console.log("Substituted");
drawProg(substitute(origAST, "x", otherAST));

Original


Other AST


Substituted


In [35]:
const origAST = Parser.parse("x + x + x");
const otherAST = Parser.parse("1 + 2");
console.log("Original");
drawProg(origAST);
console.log("Other AST");
drawProg(otherAST);
console.log("Substituted");
drawProg(substitute(origAST, "x", otherAST));

Original


Other AST


Substituted


In [36]:
const origAST = Parser.parse("x");
const otherAST = Parser.parse("1 + 4 / 2");
console.log("Original");
drawProg(origAST);
console.log("Other AST");
drawProg(otherAST);
console.log("Substituted");
drawProg(substitute(origAST, "x", otherAST));

Original


Other AST


Substituted


In [37]:
const origAST = Parser.parse("(λx => x)");
const otherAST = Parser.parse("1");
console.log("Original");
drawProg(origAST);
console.log("Other AST");
drawProg(otherAST);
console.log("Substituted");
drawProg(substitute(origAST, "x", otherAST));

Original


Other AST


Substituted


In [38]:
const origAST = Parser.parse("(λy => x)");
const otherAST = Parser.parse("1");
console.log("Original");
drawProg(origAST);
console.log("Other AST");
drawProg(otherAST);
console.log("Substituted");
drawProg(substitute(origAST, "x", otherAST));

Original


Other AST


Substituted


#### Substitution can be used to define function calls

```ts
((x) => x + 1)(2) = 2
```

or

```ts
((x) => x + 1)(2) = substitute(x + 1, x, 2)
```

In [39]:
function interpret(e: E.Expr): Value {
    switch (e.tag) {
        case "NUMBER": {
            return T.mkNumericConstant(e.value);
        }
        case "BINARY": {                
            return interpretBinop(interpret(e.left), e.operator, interpret(e.right));
        }
        case "CONDITIONAL": {
            const condVal = interpret(e.condExpr);
            if (condVal.tag === "NUMBER" && condVal.value !== 0) {
                return interpret(e.thenExpr);
            } else {
                return interpret(e.elseExpr);
            }
        }
        case "FUNCTION": { // New case 1, a function is already a value.
            return E.mkFunctionExpr(e.parameter, e.body);
        }
        case "IDENTIFIER": { // New case 2, all variables should have already been substituted away
            throw Error(`Cannot find name ${e.name} in scope}`);
        }
        case "CALL": { // New case 3
            const func = interpret(e.func);
            if (func.tag === "FUNCTION") {
                const arg = interpret(e.argument);
                return interpret(substitute(func.body, func.parameter, arg));
            } else {
                throw Error(`Cannot apply ${func} to ${e.argument}`);
            }
        }

        default: {
            throw Error("Shouldn't happen");
        }
    }
}

In [40]:
const inputAST = Parser.parse("(λx => 1 + x)(1+1)");
const outputAST = interpret(inputAST);
console.log("Input");
drawProg(inputAST);
console.log("Output");
drawProg(outputAST);

Input


Output


In [41]:
const inputAST = Parser.parse("(λx => 1 + 1 + 1)(λx => 1 + 1 + 1)");
const outputAST = interpret(inputAST);
console.log("Input");
drawProg(inputAST);
console.log("Output");
drawProg(outputAST);

Input


Output


In [42]:
const inputAST = Parser.parse("1 ? (λx => 1 + 1 + 1)(λx => 1 + 1 + 1) : 1 + (λx => 1)");
const outputAST = interpret(inputAST);
console.log("Input");
drawProg(inputAST);
console.log("Output");
drawProg(outputAST);

Input


Output


In [43]:
const inputAST = Parser.parse("1 - 1 ? (λx => 1 + 1 + 1)(λx => 1 + 1 + 1) : 1 + (λx => 1)");
const outputAST = interpret(inputAST);
console.log("Input");
drawProg(inputAST);
console.log("Output");
drawProg(outputAST);

evalmachine.<anonymous>:46
        throw Error(`Attempting ${leftVal} ${op} ${rightVal}`);
        ^

Error: Attempting [object Object] + [object Object]
    at Proxy.interpretBinop (evalmachine.<anonymous>:46:15)
    at interpret (evalmachine.<anonymous>:9:28)
    at Proxy.interpret (evalmachine.<anonymous>:17:24)
    at evalmachine.<anonymous>:5:27
    at evalmachine.<anonymous>:12:3
    at sigintHandlersWrap (node:vm:268:12)
    at Script.runInThisContext (node:vm:127:14)
    at Object.runInThisContext (node:vm:305:38)
    at Object.execute (/Users/dehuang/Documents/teaching/csc600/f22/lectures/node_modules/tslab/dist/executor.js:162:38)
    at JupyterHandlerImpl.handleExecuteImpl (/Users/dehuang/Documents/teaching/csc600/f22/lectures/node_modules/tslab/dist/jupyter.js:219:38)


### That's It!

- We've written a baby interpreter for LambdaTS.
- A "real" interpreter won't use substitution but **closures** for efficiency.
- In particular, you might imagine that substitution can make small terms really large.

### Aside: Closures

```ts
type Closure = {
    tag: "CLOSURE",               // A function has no more computation left to be run.
    func: E.FunctionExpr,         // Of course, this function can be called in the future.
    env: {[x: string]: E.Expr},   // An environment is a map from variables to expressions/values.
};
```

### Turing-Complete?

Litmus test for Turing-completeness: can you write an infinite loop?

In [44]:
// Y-combinator gives us unbounded recursion
const Y = (f) => ((x) => (y) => f(x(x))(y)) ((a) => (b) => f(a(a))(b));

In [45]:
// Factorial function
const fact = Y((f) => (n) => n ? n*f(n - 1) : 1)
fact(5)

120


In [46]:
const inputAST = Parser.parse("((λf => (λx => λy => f(x(x))(y)) (λa => λb => f(a(a))(b))) (λfact => λn => n ? n*fact(n - 1) : 1))(5)");
const outputAST = interpret(inputAST);
console.log("Input");
drawProg(inputAST);
console.log("Output");
drawProg(outputAST);

Input


Output


## Summary

1. We saw how to write an interpreter for lambdaTS. This formalizes the distinction between **values** and **expressions**.
2. This code is similar to what would be implemented by Node.js "underneath the hood".
3. Conceptually, can write an interpreter for any programming language using the ideas in this lecture.
4. We saw that LambdaTS was Turing complete.
5. Challenge 1: how would you rewrite the interpreter to use closures?
6. Challenge 2: how would you add state, classes, concurrency? How would you modify the interpreter to be lazy?